##### Multiclass Classification using Logistic Regression
Data : 5000 images containing digits (each image divided into 20 x 20 matrix representation)

###### Importing Libraries 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sp #for loading .mat file
from scipy.optimize import minimize
import pandas as pd
%matplotlib inline

###### Loading data from .mat file 

In [2]:
data = sp.loadmat("ex3data1.mat")
data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

###### Checking size of the required data 

In [3]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

###### Defining sigmoid and Cost functions 

In [4]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))


def cost(theta, x, y, rate):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    a = np.multiply(y, np.log(sigmoid(x * theta.T)))
    b = np.multiply((1-y), np.log(1 - sigmoid(x * theta.T)))
    reg = (rate/(2*len(x))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(-(a+b))/len(x) + reg

###### Defining gradient with regularization

In [5]:
def gradient(theta, x, y, rate):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    
    n = int(theta.ravel().shape[1])
    e = sigmoid(x * theta.T) - y
    
    grad = ((x.T * e)/len(x)).T + (rate/len(x))*theta
    
    # since first grad shouldn't be regularized
    grad[0,0] = np.sum(np.multiply(e, x[:,0]))/len(x)
    
    return np.array(grad).ravel() #flatten the value of gradients

###### Defining the one-vs-All method along with optimization  

In [6]:
def onevsall(x, y, num, rate):
    m = x.shape[0] # number of rows
    n = x.shape[1] # number of columns (parameters) 
    
    totalTheta = np.zeros((num, n+1)) # Shape of Theta (+1 for the theta[0]) 
    
    x = np.insert(x, obj=0, values=np.ones(m), axis=1)
    
    # since in y taken from .mat file, labels are from 1-10, not from 0-9
    for i in range(1, num+1):
        print("Carrying out for label {}".format(i))
        theta = np.zeros(n+1)
        newY = np.array([1 if label == i else 0 for label in y])
        newY = np.reshape(newY,(m,1))
        
        # Minimizing the function; jac means jacobian(gradient) [if available] 
        fmin = minimize(fun=cost, x0=theta, args=(x, newY, rate), method='TNC', jac=gradient)
        # fmin returns 3 values, first one, labelled 'x' are the optimized values 
        totalTheta[i-1,:] = fmin.x
        # print(totalTheta.shape)
        # print(newY)
        
    return totalTheta
    
    

###### Getting value of theta for all test cases 

In [7]:
print(np.unique(data['y']))

totalTheta = onevsall(data['X'], data['y'], 10, 0.1)
print(totalTheta)

[ 1  2  3  4  5  6  7  8  9 10]
Carrying out for label 1
Carrying out for label 2
Carrying out for label 3
Carrying out for label 4
Carrying out for label 5
Carrying out for label 6
Carrying out for label 7
Carrying out for label 8
Carrying out for label 9
Carrying out for label 10
[[-3.07440547e+00  0.00000000e+00  0.00000000e+00 ...  6.82476252e-03
   9.33758817e-11  0.00000000e+00]
 [-3.73227557e+00  0.00000000e+00  0.00000000e+00 ...  2.32763652e-02
  -2.55277369e-03  0.00000000e+00]
 [-5.71196765e+00  0.00000000e+00  0.00000000e+00 ... -6.38573453e-05
  -3.68653885e-07  0.00000000e+00]
 ...
 [-9.12654984e+00  0.00000000e+00  0.00000000e+00 ... -6.15416819e-04
   6.94572108e-05  0.00000000e+00]
 [-5.62646195e+00  0.00000000e+00  0.00000000e+00 ... -1.12459040e-02
   8.55763510e-04  0.00000000e+00]
 [-8.06203608e+00  0.00000000e+00  0.00000000e+00 ... -3.52728443e-05
   9.65615150e-07  0.00000000e+00]]


###### Defining the prediction function that predicts probabiltiy of every class for each input of test case 

In [8]:
def predictAll(x, Totaltheta):
    m = x.shape[0]
    n = x.shape[1]
    num = Totaltheta.shape[0]

    # Inserting ones to match shape
    x = np.insert(x, 0, values=np.ones(m), axis=1)

    # convert to matrices
    x = np.matrix(x)
    Totaltheta = np.matrix(Totaltheta)

    # compute the class probability for each class on each training instance
    h = sigmoid(x * Totaltheta.T)

    # create array of the index with the maximum probability
    maxProb = np.argmax(h, axis=1)

    # since labelling start from 1, uptill 10
    maxProb = maxProb + 1

    return maxProb

###### Calling prediction function and calculation the accuracy of the model 

In [9]:
predictedY = predictAll(data['X'], totalTheta)  
correct = [1 if a == b else 0 for (a, b) in zip(predictedY, data['y'])]  
acc = (sum(map(int, correct)) / float(len(correct)))  
print('accuracy of model = {0}%'.format(acc * 100))


accuracy of model = 96.46000000000001%
